<center>
<img src="../images/fscampus_small2.png" width="1200"/>
</center>

<center>

# Investments

***Finance 2 - BFIN***

**Dr. Omer Cayirli**

Lecturer in Empirical Finance

omer.cayirli@vgu.edu.vn
</center>

---

## Lecture 06

---


### Outline

*   Index Models and The Capital Asset Pricing Model
    
    *   The Single-Index Model
    
    *   The Capital Asset Pricing Model

---



### A Single-Factor Market

*   Advantages
    *   Reduces the number of inputs for diversification
        *   Suppose your security analysts can thoroughly analyze 50 stocks. This means that your input list will include the following:
            
            [n = 50 estimates of expected returns] + [n = 50 estimates of variances]

            $(n^2 – n)/2$ = 1,225 estimates of covariances $\quad \Rightarrow \quad$ 1,325 total estimates
    

*   Model

    $r_i = E(r_i) + \text{unanticipated surprise}$

    $r_i = E(r_i) + \beta_i m + e_i$

---



### A Single-Factor Market

*   Regression equation:
    $$R_i(t) = \alpha_i + \beta_i R_M(t) + e_i(t)$$
*   Expected return-beta relationship:
    $$E(R_i) = \alpha_i + \beta_i E(R_M)$$
*   Variance = Systematic risk + Firm-specific risk:
    $$\sigma_i^2 = \beta_i^2 \sigma_M^2 + \sigma^2(e_i)$$
*   Covariance = Product of betas $\times$ Market risk:
    $$\text{Cov}(r_i, r_j) = \beta_i \beta_j \sigma_M^2$$
*   Correlation =
    $$\text{Corr}(r_i, r_j) = \frac{\beta_i \beta_j \sigma_M^2}{\sigma_i \sigma_j} = \frac{\beta_i \sigma_M \beta_j \sigma_M}{\sigma_i \sigma_M \sigma_j \sigma_M}$$
    $$= \text{Corr}(r_i, r_m) \times \text{Corr}(r_j, r_m)$$

---



### A Single-Factor Market

*   Some securities will be more sensitive than others to macroeconomic shocks.

*   The variance of returns attributable to the marketwide factor is called the systematic risk of the security.

*   The index model assumes that firm-specific surprises are mutually uncorrelated:
    
    *   The only source of covariance between any pair of securities is their common dependence on the market return.
    
    $$\text{Cov}(r_i, r_j) = \beta_i \beta_j \sigma_M^2$$
    
    $$\text{Corr}(r_i, r_j) = \text{Corr}(r_i, r_m) \times \text{Corr}(r_j, r_m)$$


---



### A Single-Factor Market

$$R_i(t) = \alpha_i + \beta_i R_M(t) + e_i(t)$$

$$E(R_i) = \alpha_i + \beta_i E(R_M)$$

*   $\alpha$ is the security's expected excess return when the market excess return is zero. It is the vertical intercept.

*   The slope of the line in the figure is the security's beta coefficient, $\beta_i$.

---



In [12]:
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import datetime
import warnings

# --- 1. UI Widgets ---
style = {'description_width': 'initial'}
stock_widget = widgets.Text(value='AMZN', description='Stock Ticker:', style=style)
market_widget = widgets.Text(value='^SP500TR', description='Market Index:', style=style)
start_date_widget = widgets.DatePicker(description='Start Date:', value=datetime.date(2010, 1, 1))
split_date_widget = widgets.DatePicker(description='Split Date:', value=datetime.date(2018, 1, 1))
end_date_widget = widgets.DatePicker(description='End Date:', value=datetime.date(2024, 12, 31))
run_button = widgets.Button(description="Run Comparative Analysis", button_style='success', icon='cogs')
output_plots = widgets.Output()
output_stats = widgets.Output()

# --- 2. Core Analysis and Display Function ---
def run_comparative_analysis(b=None):
    with output_plots: clear_output(wait=True)
    with output_stats: clear_output(wait=True)

    stock_ticker = stock_widget.value.upper()
    market_ticker = market_widget.value.upper()
    start, split, end = start_date_widget.value, split_date_widget.value, end_date_widget.value
    
    if not (stock_ticker and market_ticker and start < split < end):
        with output_plots: display(HTML("<p style='color:red;'>Error: Provide valid tickers and ensure Start < Split < End.</p>")); return

    with output_plots: print(f"Downloading data for {stock_ticker} and {market_ticker} from {start} to {end}...")
    try:
        tickers = [stock_ticker, market_ticker]
        data = yf.download(tickers, start=start, end=end, interval='1mo', auto_adjust=True, progress=False)['Close']
        data = data.rename(columns={col: col.upper() for col in data.columns})
        if stock_ticker not in data.columns or market_ticker not in data.columns or data[stock_ticker].isnull().all() or data[market_ticker].isnull().all():
            raise ValueError(f"Could not retrieve valid data for one or both tickers.")
        returns = data.pct_change().dropna(how='all')
        
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", FutureWarning)
            ff = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start=start, end=end)[0]
        rf = ff['RF'] / 100
        
        returns.index = returns.index.to_period('M').to_timestamp()
        if isinstance(rf.index, pd.PeriodIndex):
             rf.index = rf.index.to_timestamp(how='start')

        merged_data = returns.join(rf, how='inner')
        if merged_data.empty: raise ValueError("No overlapping data found between returns and risk-free rate after alignment.")
        merged_data.columns = [stock_ticker, market_ticker, 'RF']
        excess_returns = merged_data[[stock_ticker, market_ticker]].subtract(merged_data['RF'], axis=0)
    except Exception as e:
        with output_plots: clear_output(wait=True); display(HTML(f"<p style='color:red;'>Data Error: {e}</p>")); return

    sub1_end = split - datetime.timedelta(days=1)
    sub1_returns = excess_returns.loc[start:sub1_end]
    sub2_returns = excess_returns.loc[split:end]
    
    models = {}
    for i, ret_df in enumerate([sub1_returns, sub2_returns]):
        # --- FIX: Ensure data is perfectly aligned before regression ---
        ret_df = ret_df[[stock_ticker, market_ticker]].dropna()
        # --- End of FIX ---

        if len(ret_df) < 10: models[f'sub{i+1}'] = None; continue
        y = ret_df[stock_ticker]; X = sm.add_constant(ret_df[market_ticker])
        models[f'sub{i+1}'] = sm.OLS(y, X).fit()

    with output_stats:
        html_stats = ""
        for i, model in enumerate([models['sub1'], models['sub2']]):
            period = f"({start.year}-{sub1_end.year})" if i == 0 else f"({split.year}-{end.year})"
            title = f"<h4>Sub-sample {i+1} {period}</h4>"
            if model is None:
                html_stats += f"<div style='flex:1; padding:0 10px;'>{title}<p style='color:orange;'>Insufficient data.</p></div>"; continue
            stats_df = pd.DataFrame({'Coefficient': model.params, 'Std. Error': model.bse, 't-stat': model.tvalues, 'p-value': model.pvalues}).rename(index={'const': 'Alpha', market_ticker: 'Beta'})
            summary_text = f"<b>R²:</b> {model.rsquared:.4f}, <b>Adj. R²:</b> {model.rsquared_adj:.4f}, <b>Obs:</b> {model.nobs:.0f}"
            html_stats += f"<div style='flex:1; padding:0 10px;'>{title}{stats_df.style.format('{:.4f}').to_html()}{summary_text}</div>"
        display(HTML(f"<div style='display:flex;'>{html_stats}</div>"))

    with output_plots:
        clear_output(wait=True)
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        if not excess_returns.empty:
            x_min, x_max = excess_returns[market_ticker].min(), excess_returns[market_ticker].max()
            y_min, y_max = excess_returns[stock_ticker].min(), excess_returns[stock_ticker].max()
            x_pad = (x_max - x_min) * 0.1; y_pad = (y_max - y_min) * 0.1
            global_xlim = (x_min - x_pad, x_max + x_pad); global_ylim = (y_min - y_pad, y_max + y_pad)
        else:
            global_xlim, global_ylim = (-0.2, 0.2), (-0.4, 0.4)

        for i, (ax, model, ret_df) in enumerate(zip([ax1, ax2], [models['sub1'], models['sub2']], [sub1_returns, sub2_returns])):
            period = f"({start.year}-{sub1_end.year})" if i == 0 else f"({split.year}-{end.year})"
            ax.set_title(f'SCL for {stock_ticker} - Sub-sample {i+1} {period}', fontsize=12)
            
            # Use the cleaned dataframe for plotting as well
            ret_df = ret_df[[stock_ticker, market_ticker]].dropna()

            if model is None:
                ax.text(0.5, 0.5, 'Insufficient Data', ha='center'); continue
            ax.scatter(ret_df[market_ticker], ret_df[stock_ticker], color='deepskyblue', s=15)
            ax.plot(ret_df[market_ticker], model.fittedvalues, color='black', linewidth=1.5)
            ax.set_xlabel(f'{market_ticker} Excess Return'); ax.set_ylabel(f'{stock_ticker} Excess Return')
            ax.axhline(0, color='black', linewidth=0.75); ax.axvline(0, color='black', linewidth=0.75)
            ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x*100:.0f}%'))
            ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y*100:.0f}%'))
            ax.grid(True, linestyle=':', alpha=0.6)
            ax.set_xlim(global_xlim); ax.set_ylim(global_ylim)
            
        plt.tight_layout(); plt.show()

# --- 3. Assemble UI ---
run_button.on_click(run_comparative_analysis)
controls = widgets.VBox([widgets.HBox([stock_widget, market_widget]), widgets.HBox([start_date_widget, split_date_widget, end_date_widget]), run_button], layout=widgets.Layout(align_items='center'))
display(widgets.VBox([controls, output_plots, output_stats]))
run_comparative_analysis(None)

---

### Opportunity Set of Risky Assets

The data below describe a three-stock financial market that satisfies the single-index model. These three stocks are the only stocks in this market.

| Stock | Capitalization | Beta | Mean Excess Return | Standard Deviation |
| :----: | -------------: | ---: | -----------------: | -----------------: |
| A     | 3,000          | 1.0  | 10.0%                | 40.0%                |
| B     | 1,940          | ???  | 2.0%                  | 30.0%                 |
| C     | 1,360          | 1.7  | 17.0%                 | 50.0%                 |

The standard deviation of the market-index portfolio is 25%.

  a. What is the beta of Stock B? What is the mean excess return of the index portfolio?

  b. What is the covariance between stock A and stock B?

  c. What is the covariance between stock B and the index?

  d. Break down the variance of stock B into its systematic and firm-specific components.

$$
\text{Cov}(r_i, r_j) = \beta_i \beta_j \sigma_M^2
$$

$$
\sigma_i^2 = \beta_i^2 \sigma_M^2 + \sigma^2(e_i)
$$

---



### Opportunity Set of Risky Assets

$$
\text{Total Capitalization} = 3000 + 1940 + 1360 = 6300
$$
$$
w_A = 3000 / 6300 = 0.4762 \quad w_B = 1940 / 6300 = 0.3079 \quad w_C = 1360 / 6300 = 0.2159
$$

Since these three stocks constitute the entire market, the market beta (the weighted average of individual betas) must be 1. We solve for the beta of Stock B that satisfies this condition.
$$
1 = w_A\beta_A + w_B\beta_B + w_C\beta_C
$$
$$
1 = (0.4762 \times 1.0) + (0.3079 \times \beta_B) + (0.2159 \times 1.7)
$$
$$
\beta_B = \frac{1 - 0.4762 - 0.36703}{0.3079} = 0.5093
$$


$$
E(R_M) = w_A E(R_A) + w_B E(R_B) + w_C E(R_C) = (0.4762 \times 10\%) + (0.3079 \times 2\%) + (0.2159 \times 17\%) = 9.05\%
$$


$$
\text{Cov}(r_A, r_B) = \beta_A \beta_B \sigma_M^2 = 1.0 \times 0.5093 \times (0.25)^2 = 0.03183
$$

$$
\text{Cov}(r_B, r_M) = \beta_B \sigma_M^2 = 0.5093 \times (0.25)^2 = 0.03183
$$


$$
\sigma_B^2 = (0.30)^2 = 0.09
$$

$$
\text{Systematic Risk} = \beta_B^2 \sigma_M^2 = (0.5093)^2 \times (0.25)^2 = 0.01621
$$

$$
\text{Firm-Specific Risk} = \sigma^2(e_B) = \sigma_B^2 - \beta_B^2 \sigma_M^2 = 0.09 - 0.01621 = 0.07379
$$

---

In [4]:
import numpy as np

# Data
cap_A, beta_A, ER_A, sigma_A = 3000, 1.0, 0.10, 0.40
cap_B, ER_B, sigma_B = 1940, 0.02, 0.30
cap_C, beta_C, ER_C, sigma_C = 1360, 1.7, 0.17, 0.50
sigma_M = 0.25

# Part (a): Weights and Beta B
total_cap = cap_A + cap_B + cap_C
w_A = cap_A / total_cap
w_B = cap_B / total_cap
w_C = cap_C / total_cap
beta_B = (1 - w_A * beta_A - w_C * beta_C) / w_B
ER_M = w_A * ER_A + w_B * ER_B + w_C * ER_C
print(f"w_A: {w_A:.4f}, w_B: {w_B:.4f}, w_C: {w_C:.4f}")
print(f"Beta_B: {beta_B:.4f}")
print(f"ER_M: {ER_M * 100:.2f}%")

# Part (b): Cov(A,B)
cov_AB = beta_A * beta_B * sigma_M**2
print(f"Cov(A,B): {cov_AB:.5f}")

# Part (c): Cov(B,M)
cov_BM = beta_B * sigma_M**2
print(f"Cov(B,M): {cov_BM:.5f}")

# Part (d): Variance Breakdown B
var_B = sigma_B**2
systematic_B = beta_B**2 * sigma_M**2
firm_specific_B = var_B - systematic_B
print(f"Var_B: {var_B:.2f}")
print(f"Systematic_B: {systematic_B:.5f}")
print(f"Firm-Specific_B: {firm_specific_B:.5f}")

w_A: 0.4762, w_B: 0.3079, w_C: 0.2159
Beta_B: 0.5093
ER_M: 9.05%
Cov(A,B): 0.03183
Cov(B,M): 0.03183
Var_B: 0.09
Systematic_B: 0.01621
Firm-Specific_B: 0.07379


---

### Index Model Regression Equation

$$
R_i(t) = \alpha_i + \beta_i R_{S\&P500}(t) + e_i(t)
$$

| Description | Symbol |
| :--- | :---: |
| The stock's expected return if the market is neutral,<br>that is, if the market's excess return, $r_M - r_f$, is zero | $\alpha_i$ |
| The component of return due to movements in the overall market in any period;<br>$\beta_i$ is the security's responsiveness to market movements | $\beta_i(r_M - r_f)$ |
| The unexpected component of return in any period due to unexpected events<br>that are relevant only to this security (firm specific) | $e_i$ |
| The variance attributable to the uncertainty of the common<br>macro-economic factor | $\beta_i^2 \sigma_M^2$ |
| The variance attributable to firm-specific uncertainty | $\sigma^2(e_i)$ |

---

In [13]:
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pandas_datareader.data as web
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output, Markdown
import datetime
import warnings

# --- 1. UI Widgets ---
style = {'description_width': 'initial'}
stock_widget = widgets.Text(value='AMZN', description='Stock Ticker:', style=style)
market_widget = widgets.Text(value='^SP500TR', description='Market Index:', style=style)
start_date_widget = widgets.DatePicker(description='Start Date:', value=datetime.date(2010, 1, 1))
end_date_widget = widgets.DatePicker(description='End Date:', value=datetime.date(2015, 12, 31))
run_button = widgets.Button(description="Run Single-Index Model", button_style='success', icon='cogs')
output_area = widgets.Output()

# --- 2. Main Analysis and Display Function ---
def run_single_index_model(b=None):
    with output_area:
        clear_output(wait=True)

        stock_ticker = stock_widget.value.upper()
        market_ticker = market_widget.value.upper()
        start, end = start_date_widget.value, end_date_widget.value
        
        if not (stock_ticker and market_ticker and start < end):
            display(HTML("<p style='color:red;'>Error: Provide valid tickers and a valid date range.</p>")); return

        print(f"Downloading data and running regression for {stock_ticker} vs. {market_ticker}...")
        try:
            tickers = [stock_ticker, market_ticker]
            data = yf.download(tickers, start=start, end=end, interval='1mo', auto_adjust=True, progress=False)['Close']
            data = data.rename(columns={col: col.upper() for col in data.columns})
            if stock_ticker not in data.columns or market_ticker not in data.columns or data[stock_ticker].isnull().all() or data[market_ticker].isnull().all():
                raise ValueError(f"Could not retrieve valid data for one or both tickers.")
            returns = data.pct_change().dropna()
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", FutureWarning)
                ff = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start=start, end=end)[0]
            rf = ff['RF'] / 100
            returns.index = returns.index.to_period('M').to_timestamp()
            if isinstance(rf.index, pd.PeriodIndex): rf.index = rf.index.to_timestamp(how='start')
            merged_data = returns.join(rf, how='inner')
            if merged_data.empty: raise ValueError("No overlapping data found.")
            merged_data.columns = [stock_ticker, market_ticker, 'RF']
            excess_returns = merged_data[[stock_ticker, market_ticker]].subtract(merged_data['RF'], axis=0)
            if excess_returns.empty or excess_returns.isnull().values.any(): raise ValueError("Data alignment resulted in empty data.")
            
            y = excess_returns[stock_ticker]
            X = sm.add_constant(excess_returns[market_ticker])
            model = sm.OLS(y, X).fit()

            clear_output(wait=True)
            
            stats_summary = {'Multiple R': np.sqrt(model.rsquared), 'R-Square': model.rsquared, 'Adjusted R-Square': model.rsquared_adj, 'Standard Error': np.sqrt(model.mse_resid), 'Observations': int(model.nobs)}
            stats_df1 = pd.DataFrame(pd.Series(stats_summary, name=''))
            
            stats_df2 = pd.DataFrame({'Coefficients': model.params, 'Standard Error': model.bse, 't-statistic': model.tvalues, 'p-value': model.pvalues}).rename(index={'const': 'Intercept', market_ticker: 'Market index'})

            # --- FIX: Define the explanatory text ---
            explanation_md = """
*   The adjusted R-square corrects for an upward bias in R-square that arises because we use the estimated values of two parameters, the slope (beta) and intercept (alpha), rather than their true, but unobservable, values.
*   The standard error of the regression is the standard deviation of the residual, e.
    *   Higher the standard errors greater the impact of firm-specific events.
"""
            # --- End of FIX ---

            display(HTML(f"<h4>Regression statistics for {stock_ticker}</h4>"))
            display(stats_df1.style.format('{:.4f}', subset=pd.IndexSlice[:'Standard Error']))
            display(stats_df2.style.format('{:.4f}'))
            display(Markdown(explanation_md)) # Display the text

        except Exception as e:
            clear_output(wait=True)
            display(HTML(f"<p style='color:red;'>An error occurred: {e}</p>"))

# --- 3. Assemble UI ---
run_button.on_click(run_single_index_model)
controls = widgets.VBox([widgets.HBox([stock_widget, market_widget]), widgets.HBox([start_date_widget, end_date_widget]), run_button], layout=widgets.Layout(align_items='center'))

display(widgets.VBox([controls, output_area]))
run_single_index_model()

---

### Index Model Regression Equation

| Ticker | Company           | Beta   | Alpha  | R-Square | Residual Std Dev | Standard Error Beta | Standard Error Alpha | Adjusted Beta |
| :----- | :---------------- | :-----: | :-----: | :-------: | :---------------: | :------------------: | :-------------------: | :------------: |
| CPB    | Campbell Soup     | 0.247  | 0.001  | 0.012    | 0.064            | 0.292               | 0.009                | 0.498         |
| MCD    | McDonald's        | 0.563  | 0.006  | 0.165    | 0.036            | 0.165               | 0.005                | 0.709         |
| SBUX   | Starbucks         | 0.581  | 0.004  | 0.112    | 0.046            | 0.212               | 0.006                | 0.721         |
| KO     | Coca-Cola         | 0.663  | -0.001 | 0.258    | 0.032            | 0.146               | 0.004                | 0.776         |
| UNP    | Union Pacific     | 0.859  | 0.005  | 0.212    | 0.047            | 0.216               | 0.006                | 0.906         |
| PFE    | Pfizer            | 0.895  | -0.000 | 0.367    | 0.033            | 0.153               | 0.005                | 0.930         |
| XOM    | ExxonMobil        | 0.920  | -0.006 | 0.342    | 0.036            | 0.166               | 0.005                | 0.946         |
| MSFT   | Microsoft         | 0.923  | 0.012  | 0.193    | 0.054            | 0.246               | 0.007                | 0.948         |
| INTC   | Intel             | 0.934  | 0.007  | 0.187    | 0.055            | 0.254               | 0.007                | 0.956         |
| GOOG   | Alphabet | 0.960  | 0.008  | 0.209    | 0.053            | 0.243               | 0.007                | 0.973         |
| DIS    | Walt Disney       | 1.288  | -0.001 | 0.496    | 0.037            | 0.169               | 0.005                | 1.192         |
| BAC    | Bank of America   | 1.357  | 0.003  | 0.270    | 0.063            | 0.291               | 0.009                | 1.238         |
| BA     | Boeing            | 1.368  | 0.011  | 0.330    | 0.055            | 0.254               | 0.007                | 1.246         |
| AMZN   | Amazon            | 1.533  | 0.019  | 0.286    | 0.069            | 0.315               | 0.009                | 1.355         |
| MRO    | Marathon Oil      | 2.632  | -0.023 | 0.314    | 0.110            | 0.506               | 0.015                | 2.088         |
|        |                   |        |        |          |                  |                     |                      |               |
| AVERAGE|                   | 1.010  | 0.003  | 0.235    | 0.057            | 0.260               | 0.008                | 1.006         |
| STD DEV |          | 0.560  | 0.009  | 0.127    | 0.025            | 0.115               | 0.003                | 0.374         |

---



In [14]:
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pandas_datareader.data as web
from scipy.optimize import minimize
from IPython.display import display, HTML, clear_output, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt
import datetime
import warnings
import io, base64

ALL_TICKERS = ['AMD', 'AMZN', 'F', 'JPM', 'NVDA', 'PFE', 'TGT', 'XOM', 'WMT', 'VZ']
start_date_widget = widgets.DatePicker(description='Start Date:', value=datetime.date(2010, 1, 1))
end_date_widget = widgets.DatePicker(description='End Date:', value=datetime.date(2024, 12, 31))
checkbox_widgets = {t: widgets.Checkbox(value=True, description=t, indent=False, layout=widgets.Layout(width='auto')) for t in ALL_TICKERS}
allow_short_widget = widgets.Checkbox(value=False, description='Allow Short Selling')
run_button = widgets.Button(description="Run Model Comparison", button_style='success', icon='cogs')
output_area = widgets.Output()

def optimize_frontier(mean_returns, cov_matrix, rf_rate, allow_short):
    if len(mean_returns) < 2: return None
    num_assets = len(mean_returns)
    initial_guess = np.array(num_assets * [1./num_assets])
    def portfolio_variance(w, cov): return w.T @ cov @ w
    def neg_sharpe(w, mean_ret, cov, rf):
        er = w.T @ mean_ret; std = np.sqrt(w.T @ cov @ w)
        return -(er - rf) / std if std > 1e-9 else 1e9
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    bounds = None if allow_short else tuple((0, 1) for _ in range(num_assets))
    opt_result = minimize(neg_sharpe, initial_guess, args=(mean_returns, cov_matrix, rf_rate), method='SLSQP', bounds=bounds, constraints=constraints)
    if not opt_result.success: return None
    opt_weights = opt_result.x
    opt_er = opt_weights @ mean_returns
    opt_std = np.sqrt(opt_weights.T @ cov_matrix @ opt_weights)
    opt_sharpe = (opt_er - rf_rate) / opt_std
    lo, hi = mean_returns.min(), mean_returns.max()
    if np.isclose(lo, hi):
        target_returns = np.linspace(lo - 1e-6, hi + 1e-6, 3)
    else:
        target_returns = np.linspace(lo, hi, 50)
    frontier_std = []
    for tr in target_returns:
        constraints_fr = [{'type': 'eq', 'fun': lambda w: np.sum(w)-1}, {'type': 'eq', 'fun': lambda w: w.T @ mean_returns - tr}]
        res = minimize(portfolio_variance, initial_guess, args=(cov_matrix,), method='SLSQP', bounds=bounds, constraints=constraints_fr)
        if res.success: frontier_std.append(np.sqrt(res.fun))
        else: frontier_std.append(np.nan)
    return {'weights': opt_weights, 'er': opt_er, 'std': opt_std, 'sharpe': opt_sharpe, 'frontier_stds': np.array(frontier_std), 'frontier_returns': target_returns}

def run_comparison(b=None):
    with output_area:
        clear_output(wait=True)
        selected_tickers = [t for t, cb in checkbox_widgets.items() if cb.value]
        allow_short = allow_short_widget.value
        market_ticker = '^SP500TR'
        start, end = start_date_widget.value, end_date_widget.value
        if len(selected_tickers) < 2 or not start < end:
            display(HTML("<p style='color:red;'>Error: Select at least 2 assets and a valid date range.</p>")); return
        print("Downloading data...")
        try:
            all_tickers_dl = list(set(selected_tickers + [market_ticker]))
            data = yf.download(all_tickers_dl, start=start, end=end, interval='1mo', auto_adjust=True, progress=False)['Close']
            returns = data.pct_change()
            if market_ticker not in returns.columns:
                raise ValueError(f"Market proxy {market_ticker} not found.")
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", FutureWarning)
                ff = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start=start, end=end)[0]
            returns.index = returns.index.to_period('M').to_timestamp()
            ff.index = ff.index.to_timestamp(how='start')
            rf_series = (ff['RF'] / 100)
            excess_returns = returns.join(rf_series).dropna()
            if excess_returns.empty: raise ValueError("No valid overlapping data after aligning returns and risk-free rate.")
            market_excess = excess_returns[market_ticker] - excess_returns['RF']
            stock_excess = excess_returns[selected_tickers].subtract(excess_returns['RF'], axis=0)
        except Exception as e:
            display(HTML(f"<p style='color:red;'>Data Error: {e}</p>")); return
        
        print("Running optimizations...")
        full_cov_matrix = stock_excess.cov()
        mean_excess_returns = stock_excess.mean()
        full_cov_results = optimize_frontier(mean_excess_returns, full_cov_matrix, 0, allow_short)

        betas, alphas, resid_vars = {}, {}, {}
        X = sm.add_constant(market_excess)
        X.columns = ['const', 'MKT']
        for stock in stock_excess.columns:
            temp_df = pd.concat([stock_excess[stock], X['MKT']], axis=1).dropna()
            if len(temp_df) < 10: continue
            y_reg = temp_df[stock]
            X_reg = sm.add_constant(temp_df['MKT'])
            model = sm.OLS(y_reg, X_reg).fit()
            alphas[stock] = model.params['const']; betas[stock] = model.params['MKT']; resid_vars[stock] = model.mse_resid
        
        betas, alphas, resid_vars = pd.Series(betas), pd.Series(alphas), pd.Series(resid_vars)
        if betas.empty:
            display(HTML("<p style='color:orange;'>Could not estimate betas for any selected stock.</p>")); return
            
        index_model_cov = pd.DataFrame(np.outer(betas, betas) * market_excess.var() + np.diag(resid_vars), index=betas.index, columns=betas.index)
        mean_excess_returns_im = alphas + betas * market_excess.mean()
        index_model_results = optimize_frontier(mean_excess_returns_im, index_model_cov, 0, allow_short)
        
        clear_output(wait=True)
        if index_model_results is None or full_cov_results is None:
            display(HTML("<p style='color:orange;'>Optimization failed. Try different assets or a longer period.</p>")); return
        
        weights_index = pd.Series(index_model_results['weights'], index=betas.index)
        weights_full = pd.Series(full_cov_results['weights'], index=stock_excess.columns).reindex(betas.index)
        weights_df = pd.DataFrame({'Index Model': weights_index, 'Full-Covariance Model': weights_full})
        
        stats_data = {'Index Model': [index_model_results['er'], index_model_results['std'], index_model_results['sharpe']], 'Full-Covariance Model': [full_cov_results['er'], full_cov_results['std'], full_cov_results['sharpe']]}
        stats_df = pd.DataFrame(stats_data, index=['Risk premium', 'Standard deviation', 'Sharpe ratio'])
        table_html = "<h4>A. Weights in Optimal Risky Portfolio</h4>" + weights_df.style.format('{:.2%}').to_html()
        table_html += "<h4>B. Portfolio Characteristics</h4>" + stats_df.style.format('{:.4f}').to_html()
        
        fig, ax = plt.subplots(figsize=(7, 6))
        m1 = np.isfinite(index_model_results['frontier_stds']); m2 = np.isfinite(full_cov_results['frontier_stds'])
        ax.plot(index_model_results['frontier_stds'][m1], index_model_results['frontier_returns'][m1], color='black', marker='d', markersize=4, label='Index Model')
        ax.plot(full_cov_results['frontier_stds'][m2], full_cov_results['frontier_returns'][m2], color='deepskyblue', marker='s', markersize=4, label='Full Covariance Model')
        ax.set_xlabel('Standard Deviation'); ax.set_ylabel('Risk Premium'); ax.set_title('Efficient Frontier Comparison')
        ax.legend(); ax.grid(True, linestyle=':', alpha=0.7)
        buf = io.BytesIO(); plt.savefig(buf, format='png'); buf.seek(0)
        img_str = base64.b64encode(buf.read()).decode('utf-8'); plt.close(fig); buf.close()
        plot_html = f'<img src="data:image/png;base64,{img_str}"/>'
        
        explanation_md = """
*   Is the index model inferior to the full-blown Markowitz model?
    *   It imposes additional assumptions that may not be fully accurate.
    *   The Markowitz model allows far more flexibility in modeling asset covariance structure.
    *   Estimating the covariances with a sufficient degree of accuracy is an issue.
"""
        
        display(HTML(f"<div style='display:flex; align-items: flex-start;'><div style='flex:1; padding-right:20px;'>{table_html}</div><div style='flex:1;'>{plot_html}</div></div>"))
        display(Markdown(explanation_md))

run_button.on_click(run_comparison)
date_controls = widgets.HBox([start_date_widget, end_date_widget])
all_checkboxes = list(checkbox_widgets.values()); midpoint = len(all_checkboxes) // 2
checkbox_grid = widgets.VBox([widgets.HBox(all_checkboxes[:midpoint]), widgets.HBox(all_checkboxes[midpoint:])])
asset_controls = widgets.VBox([widgets.Label("Select Assets:"), checkbox_grid, allow_short_widget])
ui = widgets.VBox([date_controls, asset_controls, run_button], layout=widgets.Layout(align_items='center'))

display(widgets.VBox([ui, output_area]))
run_comparison(None)

---

### The Capital Asset Pricing Model (CAPM)

**Portfolio Theory vs. CAPM**
*   **Mean-Variance Analysis (Portfolio Theory):**
    *   Tells us how an investor can select an optimal risky portfolio (the tangency portfolio) given expected returns, variances, and covariances.
    *   It's a model of how to choose portfolios but doesn't determine what equilibrium asset prices or expected returns should be.
*   **CAPM:**
    *   An equilibrium model that characterizes the risk-return combinations of securities that must hold if all investors are mean-variance optimizers and markets are in equilibrium.
    *   Equilibrium implies no investor wishes to change their strategy, and all assets are held (markets clear).
    *   If everyone holds an efficient portfolio, what must asset prices (and thus expected returns) be?

*   Modern Portfolio Theory states that all investors, regardless of risk aversion, will choose to hold a combination of just two portfolios:
    *   The risk-free asset
    *   The same optimal risky portfolio (the tangency portfolio)
*   In market equilibrium, if all investors hold this same tangency portfolio of risky assets, then this portfolio must, in aggregate, be the market portfolio.
    *   The market portfolio consists of all risky assets (stocks, bonds, real estate, human capital, etc.) held in proportion to their total market value.
*   The market portfolio is the truly efficient risky portfolio, and its risk premium becomes the benchmark for pricing all other risky assets.

---

### The Capital Asset Pricing Model (CAPM)

**The CAPM Assumptions**
*   All investors are rational mean-variance optimizers with identical planning horizons.
*   Investors can borrow or lend unlimited amounts at a fixed risk-free rate ($r_f$).
*   Markets are perfectly competitive: no individual investor can influence security prices.
*   All assets are tradable and perfectly divisible.
*   No taxes, transaction costs, or restrictions on short selling.
*   Investors have homogeneous expectations. 
    * They agree on the estimates of expected returns, variances, and covariances for all assets.

**The CAPM: Main Result**

The expected return on any security $i$, $E(R_i)$, is determined by:
$$ E[R_i] = r_f + \beta_i \times (E[R_{Mkt}] - r_f) \quad \Rightarrow \quad \beta_i (E(R_{Mkt}) - r_f) = \text{Risk premium for security i}$$
Where:
*   $E(R_{Mkt})$ = Expected return of the market portfolio
*   $E(R_{Mkt}) - r_f$ = Market Risk Premium (MRP)
*   $\beta_i$ = Beta of security $i$

**Beta ($\beta_i$)**
Beta measures the systematic risk of security $i$:
$$ \beta_i = \frac{\text{Cov}(R_i, R_{Mkt})}{\text{Var}(R_{Mkt})} = \frac{\sigma_i \rho_{i,Mkt}}{\sigma_{Mkt}} $$

---

### The Capital Asset Pricing Model: Security Market Line (SML)

*   Plots the expected return of assets against their beta.

*   The CAPM states that all assets must be on the SML.


<img src = "../images/slide_7/pic_3.png" width="540">

*   SML vs CML
    *   SML
        *   Shows systematic risk only
        *   Every security / portfolio lies on the SML
    *   CML
        *   Shows the total risk (systematic + unsystematic)
        *   Only two securities lie on the CML
            *   Risk-free asset and the market portfolio

---



### The Capital Asset Pricing Model
<img src = "..\images\slide_7\pic_4.png">

---



### The Capital Asset Pricing Model

<img src = "..\\Pictures\\slide_7\\pic_5.png">

---



### The Capital Asset Pricing Model

*   The risk premium on the market portfolio is proportional to its risk and the degree of risk aversion:
    $$
    E(R_M) = \bar{A} \sigma_M^2
    $$
    *   An individual security's risk premium is a function of:
        *   Its contribution to the risk of the market portfolio
        *   The covariance of returns with the assets that make up the market portfolio
    $$
    \sum_{i=1}^{n} w_i \text{Cov}(R_i, R_{GE}) = \text{Cov}\left(\sum_{i=1}^{n} w_i R_i, R_{GE}\right)
    $$
    
    $$
    \frac{\text{GE's contribution to risk premium}}{\text{GE's contribution to variance}} = \frac{w_{GE} E(R_{GE})}{w_{GE} \text{Cov}(R_{GE}, R_M)} = \frac{E(R_{GE})}{\text{Cov}(R_{GE}, R_M)}
    $$
    
    $$
    \frac{\text{Market risk premium}}{\text{Market variance}} = \frac{E(R_M)}{\sigma_M^2}
    $$
    
    $$
    E(R_{GE}) = \frac{\text{Cov}(R_{GE}, R_M)}{\sigma_M^2} E(R_M)
    $$
    
    $$
    E(r_{GE}) = r_f + \beta_{GE}[E(r_M) - r_f]
    $$

---

### The Capital Asset Pricing Model

*   CAPM holds for the overall portfolio because:
    $$E(r_P) = \sum_k w_k E(r_k)$$
    $$\beta_P = \sum_k w_k \beta_k$$
*   This also holds for the market portfolio:
    $$E(r_M) = r_f + \beta_M [E(r_M) - r_f]$$

---



### The Capital Asset Pricing Model

<img src = "..\\Pictures\\slide_7\\pic_8.png">

*   An asset that is not priced according to the CAPM will not line up on the SML.
*   The difference between its actual risk premium and its risk premium predicted by the CAPM is called the asset's alpha:
    $$E(r_i) - r_f = \alpha_i + \beta_i[E(r_M) - r_f]$$

---



### Cost of Capital

* The Equity Cost of Capital
    * Best expected return available in the market on investments with similar risk.
    * Under the CAPM?
        * Investments have similar risk if they have the same sensitivity to market risk, as measured by their beta with the market portfolio.

* The Market Portfolio
    * Value-weighted vs. equal-weighted
    * S&P 500, DJIA, NDQ, S&P 1500 Composite, Russel 2000, Wilshire 5000.

* The Market Risk Premium
    * Risk-Free Rate: Maturity vs. investment horizon
    * The Risk Premium
        * Historical
        * Expected
<style>
  .returns-table {
    border-collapse: collapse;
    width: auto; /* Or specify a width like 600px */
    margin: 1em 0;
    font-family: Arial, sans-serif; /* Common sans-serif font */
    color: white; /* Default text color for data rows */
    background-color: #003366; /* Dark blue background for the table body */
    border: 1px solid #FFFFFF; /* White border for table */
  }
  .returns-table th, .returns-table td {
    border: 1px solid #FFFFFF; /* White cell borders */
    padding: 8px 10px; /* More padding */
    text-align: left;
  }
  .returns-table th { /* Styling for the header row */
    background-color: #D3D3D3; /* Light grey background for headers */
    color: #333333; /* Dark grey text for headers for contrast, or white if preferred */
    /* If you want white text on light grey like the image:
    color: #FFFFFF; (might need a slightly darker grey for background for readability)
    Let's try to match the image's subtle header text:
    */
    color: #4A4A4A; /* A slightly darker grey for text on light grey bg, or adjust as needed */
    font-weight: bold; /* Headers are often bold */
    text-align: center;
  }
  .returns-table td.metric {
    font-weight: normal; /* Normal weight for metric descriptions */
  }
  .returns-table td.value {
    text-align: right;
    font-weight: bold; /* Values are bold */
  }
  .returns-table tr.premium td.metric { /* For the "Equity Risk Premium" rows */
    font-weight: bold;
  }
  .returns-table caption {
    caption-side: bottom;
    font-size: 0.9em;
    color: #A9A9A9; /* Lighter grey for source, assuming it's outside the dark blue box */
    padding-top: 8px;
    text-align: center;
  }
</style>

<table class="returns-table">
  <thead>
    <tr>
      <th>Metric</th>
      <th>Average Annual Return (1928–2024)</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td class="metric">S&P 500 Total Return (incl. dividends)</td>
      <td class="value">11.79%</td>
    </tr>
    <tr>
      <td class="metric">3-Month Treasury Bill Return</td>
      <td class="value">3.36%</td>
    </tr>
    <tr class="premium">
      <td class="metric">Equity Risk Premium vs. T-Bill</td>
      <td class="value">8.43%</td>
    </tr>
    <tr>
      <td class="metric">10-Year Treasury Bond Return</td>
      <td class="value">4.79%</td>
    </tr>
    <tr class="premium">
      <td class="metric">Equity Risk Premium vs. 10-Yr Bond</td>
      <td class="value">7.00%</td>
    </tr>
  </tbody>
</table>

[Source: NYU Stern, Historical Returns (Damodaran)](https://pages.stern.nyu.edu/~adamodar/pc/datasets/histretSP.xls)

---

---



### The Capital Asset Pricing Model

*   Beta Estimation
    *   Using Historical Returns
        *   What is the assumption?
    *   Best-fitting line
        *   Beta corresponds to the slope of the best-fitting line in the plot of the security's excess returns vs. the market excess return.
    *   Linear Regression
    $$ (R_i - r_f) = \alpha_i + \beta_i(R_M - r_f) + e_i $$

---



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_datareader.data as web
import statsmodels.api as sm
import datetime
import warnings
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# --- Style and Layout for Widgets ---
style_desc_date = {'description_width': '80px'}
layout_date_picker = widgets.Layout(width='220px')
layout_button_run = widgets.Layout(width='auto', min_width='200px', margin='10px 0 0 0')

# --- Input Widgets for Dates ---
start_date_widget = widgets.DatePicker(
    description='Start Date:', value=datetime.date(2015, 1, 1),
    style=style_desc_date, layout=layout_date_picker
)
end_date_widget = widgets.DatePicker(
    description='End Date:', value=datetime.date(2024, 12, 31),
    style=style_desc_date, layout=layout_date_picker
)
run_button = widgets.Button(
    description="Fetch Data & Plot Betas", button_style='success', icon='cogs',
    layout=layout_button_run
)

# --- Output Widgets ---
output_plots_container = widgets.Output()
output_regression_stats_container = widgets.Output()

# --- CSS for Statsmodels Summary Table ---
STATSMODELS_TABLE_CSS = """
<style>
    div.regression-summary-wrapper table {{
        width: auto !important; margin-left: 0px !important; margin-right: auto !important;
        border-collapse: collapse !important; font-family: Arial, sans-serif !important;
        font-size: 0.85em !important; text-align: right !important;
    }}
    div.regression-summary-wrapper th, div.regression-summary-wrapper td {{
        border: 1px solid #D3D3D3 !important; padding: 4px 6px !important;
    }}
    div.regression-summary-wrapper th {{
        background-color: #f0f0f0 !important; text-align: center !important;
    }}
    div.regression-summary-wrapper caption {{
        margin-bottom: 0.5em !important; font-weight: bold !important; fontsize: 1.1em !important;
    }}
    div.regression-summary-wrapper table.simpletable td:first-child {{ text-align: left !important; }}
    div.regression-summary-wrapper table.simpletable:first-of-type th {{ background-color: #E8E8E8 !important; }}
    div.regression-summary-wrapper table.simpletable:first-of-type td {{ min-width: 80px; }}
    div.regression-summary-wrapper table.simpletable:nth-of-type(2) th {{ background-color: #F5F5F5 !important; }}
    div.regression-summary-wrapper table.simpletable:last-of-type th {{ background-color: #E8E8E8 !important; }}
    div.regression-summary-wrapper table.simpletable:last-of-type td {{ min-width: 80px; }}
</style>
"""

# --- Main Function to Fetch Data, Calculate, and Plot ---
def fetch_calculate_and_plot(b=None):
    with output_plots_container: clear_output(wait=True)
    with output_regression_stats_container: clear_output(wait=True)

    start_date_dt = datetime.datetime.combine(start_date_widget.value, datetime.time.min)
    end_date_dt = datetime.datetime.combine(end_date_widget.value, datetime.time.max)

    if start_date_dt >= end_date_dt:
        with output_plots_container: display(HTML("<p style='color:red;'>Error: Start date must be before end date.</p>"))
        return

    stock_symbols = ['AMZN', 'CSCO', 'XOM', 'TSLA']
    market_ticker = '^SP500TR'  # Corrected ticker
    all_tickers = stock_symbols + [market_ticker]
    adj_close_df = pd.DataFrame()
    
    with output_plots_container:
        try:
            print(f"Downloading data for: {all_tickers} from {start_date_dt.date()} to {end_date_dt.date()}...")
            # RULE 1.1: Use auto_adjust=True for adjusted prices
            data_downloaded = yf.download(
                all_tickers, 
                start=start_date_dt, 
                end=end_date_dt, 
                interval='1mo', 
                auto_adjust=True,  # Use adjusted prices
                group_by='ticker', 
                progress=False,
                actions=False # RULE 1.1: actions=False
            )
            if data_downloaded.empty: raise ValueError("yf.download returned an empty DataFrame.")
            df_list = []
            for ticker in all_tickers:
                try: 
                    if ticker in data_downloaded.columns.levels[0]:
                        # With auto_adjust=True, 'Close' is already adjusted
                        if 'Close' in data_downloaded[ticker].columns and not data_downloaded[ticker]['Close'].isnull().all():
                            series_to_add = data_downloaded[ticker]['Close'].rename(ticker)
                        else: continue
                        df_list.append(series_to_add)
                except Exception: pass
            if not df_list: raise ValueError("No valid data extracted for any ticker.")
            adj_close_df = pd.concat(df_list, axis=1)
            if adj_close_df.empty: raise ValueError("adj_close_df empty after concat.")
            # RULE 1.2: NEVER forward-fill price data. Use dropna(how='all').
            # Also, ensure market ticker is present before cleaning.
            if market_ticker not in adj_close_df.columns:
                 raise ValueError(f"Market ticker {market_ticker} not found in data.")
            adj_close_df = adj_close_df.dropna(how='all') # Corrected from .ffill()
            # RULE 1.2: Ensure market data exists after cleaning
            if adj_close_df.empty or adj_close_df[market_ticker].isnull().all():
                raise ValueError(f"Market ticker {market_ticker} data missing after cleaning.")
            # RULE 1.2: Do not use dropna(axis=0, how='any') here for price data before returns calculation.
            # The market data check is sufficient for the next step.
            
        except Exception as e:
            clear_output(wait=True); display(HTML(f"<p style='color:red;'>Data Acquisition Error: {e}</p>"))
            return

        # RULE 1.3: Calculate returns
        total_returns_df = adj_close_df.pct_change()
        # RULE 1.3: Drop rows where ALL columns are NaN
        total_returns_df = total_returns_df.dropna(how='all')
        # RULE 1.3: Filter columns with insufficient data (e.g., >= 12 months)
        min_obs = 12
        total_returns_df = total_returns_df.loc[:, total_returns_df.notna().sum() >= min_obs]
        # RULE 1.3: Check if we have enough assets left
        if total_returns_df.shape[1] < 2:
            clear_output(wait=True); display(HTML("<p style='color:red;'>Not enough data for analysis (fewer than 2 assets with >=12 monthly returns).</p>"))
            return
        # RULE 1.3: Check if market ticker is still present after filtering
        if market_ticker not in total_returns_df.columns:
            clear_output(wait=True); display(HTML(f"<p style='color:red;'>Market ticker {market_ticker} has insufficient data after filtering.</p>"))
            return
        

        rf_series = pd.Series(dtype='float64')
        try:
            print("Downloading Fama/French risk-free rate data...")
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", FutureWarning)
                # RULE 5.1: Use end-of-period risk-free rate
                ff_start = start_date_dt
                ff_end = end_date_dt
                ff_factors_monthly = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start=ff_start, end=ff_end)[0]
            if ff_factors_monthly.empty: raise ValueError("F/F monthly data empty.")
            # RULE 5.1: Use most recent (end-of-period) rate
            rf_rate = (ff_factors_monthly['RF'] / 100).iloc[-1]
            # Create a series with the single rate value, aligned to the returns index
            # This is a simplification; ideally, we'd align monthly RF rates to the return dates.
            # For this context, using a single rate is acceptable if documented.
            rf_aligned = pd.Series(rf_rate, index=total_returns_df.index)
        except Exception as e_ff:
            print(f"Warning: F/F download error: {e_ff}. Using rf=0%.")
            # RULE 5.2: Acceptable fallback for RF rate
            rf_aligned = pd.Series(0.0, index=total_returns_df.index)

        market_ex_returns = pd.Series(dtype='float64')
        stock_ex_returns_df = pd.DataFrame()
        if not total_returns_df.empty and not rf_aligned.isnull().all() and market_ticker in total_returns_df.columns:
            market_ex_returns_temp = total_returns_df[market_ticker] - rf_aligned
            stock_ex_returns_df_temp = total_returns_df.drop(columns=[market_ticker], errors='ignore').subtract(rf_aligned, axis=0)
            market_ex_returns = market_ex_returns_temp.dropna()
            stock_ex_returns_df = stock_ex_returns_df_temp # Keep all stock columns for now
            if not market_ex_returns.empty:
                # RULE 1.5: For per-asset analysis (like plotting betas), align data independently per asset
                # This avoids forcing a common intersection which is not needed for individual regressions.
                final_market_ex_returns = market_ex_returns
                temp_stock_dfs = {}
                for stock_col in list(stock_ex_returns_df.columns): 
                    if stock_col not in stock_symbols: continue # Only process defined stock_symbols
                    stock_series = stock_ex_returns_df[stock_col].dropna()
                    # Find intersection of this stock's data with the market data
                    intersect_idx = final_market_ex_returns.index.intersection(stock_series.index)
                    if len(intersect_idx) >= 10: # Ensure enough data for this specific stock
                         temp_stock_dfs[stock_col] = stock_series.loc[intersect_idx]
                         # Also re-align the market returns for this specific stock's available dates
                         # This ensures the regression uses the maximum available data for each stock.
                         temp_stock_dfs[f"{stock_col}_market"] = final_market_ex_returns.loc[intersect_idx]
                    else:
                         print(f"Info: Insufficient overlapping data for {stock_col} with market. Dropping for plots.")
                
                if not temp_stock_dfs: # If no stocks remain
                    stock_ex_returns_df = pd.DataFrame() # Make it empty
                    market_ex_returns = pd.Series(dtype='float64') # Also empty market if no stocks
                else:
                    # Create a structure to hold data for each stock pair (stock, market)
                    stock_market_data = {}
                    for key, value in temp_stock_dfs.items():
                        # Find the original stock symbol from the key
                        original_stock = key if not key.endswith('_market') else key[:-7]
                        if original_stock not in stock_market_data:
                            stock_market_data[original_stock] = {}
                        if key.endswith('_market'):
                            stock_market_data[original_stock]['market'] = value
                        else:
                            stock_market_data[original_stock]['stock'] = value
                    # The main stock_ex_returns_df and market_ex_returns are now used as base,
                    # but individual regressions will use their own aligned data.
                    # For the main plot loop, we will iterate through stock_market_data.
                    # Re-assign for clarity in the loop
                    final_stock_market_data = stock_market_data
            else: # If market_ex_returns became empty
                stock_ex_returns_df = pd.DataFrame()
        else:
            clear_output(wait=True); display(HTML("<p style='color:red;'>Critical data missing for excess return calc.</p>")); return
        
        clear_output(wait=True) 
        if 'final_stock_market_data' not in locals() or not final_stock_market_data:
            display(HTML("<p style='color:red;'>Plotting aborted: No valid stock data remains after processing for the selected period.</p>")); return

        plt.style.use('default') # Seaborn-v0_8 is deprecated
        plt.rcParams['axes.grid'] = True; plt.rcParams['grid.linestyle'] = ':'; plt.rcParams['grid.alpha'] = 0.3
        plt.rcParams['axes.edgecolor'] = '#DDDDDD'; plt.rcParams['axes.linewidth'] = 0.6
        plt.rcParams['font.sans-serif'] = ['Arial', 'Helvetica', 'DejaVu Sans']

        plotable_stock_symbols = list(final_stock_market_data.keys())
        num_stocks_to_plot = len(plotable_stock_symbols)

        ncols = 2; nrows = (num_stocks_to_plot + ncols - 1) // ncols
        if nrows == 0: # Handle case where no stocks are left to plot
            display(HTML("<p style='color:orange;'>No stocks have sufficient data for plotting after final alignment.</p>"))
            return

        subplot_width = 7.5 
        subplot_height = 6.0 
        fig_width = subplot_width * ncols
        fig_height = subplot_height * nrows
        
        fig, axes = plt.subplots(nrows, ncols, figsize=(fig_width, fig_height), squeeze=False)
        axes = axes.flatten()
        
        line_color = '#8B0000'; scatter_color = '#1f77b4'
        common_x_limits = (-0.30, 0.30) 
        
        regression_summaries_html_content = ""

        for i, ticker_stock in enumerate(plotable_stock_symbols): # Iterate over actual plotable symbols
            ax = axes[i]
            # Data should already be aligned for this specific stock
            current_stock_ex_ret = final_stock_market_data[ticker_stock]['stock']
            current_market_ex_ret_aligned = final_stock_market_data[ticker_stock]['market']

            if len(current_market_ex_ret_aligned) < 10: # Should have been caught earlier, but as a safeguard
                ax.text(0.5, 0.5, f'{ticker_stock}\nInsufficient data', ha='center', va='center', fontsize=10); ax.set_xticks([]); ax.set_yticks([]); continue

            X = sm.add_constant(current_market_ex_ret_aligned.values)
            y = current_stock_ex_ret.values
            try:
                model = sm.OLS(y, X).fit()
                alpha_est, beta_est = model.params[0], model.params[1]
                r_squared = model.rsquared
                summary_title = f"{ticker_stock} vs {market_ticker.replace('^','')}"
                model_summary_html = model.summary(title=summary_title).as_html()
                regression_summaries_html_content += f"<div class='regression-summary-wrapper'>{model_summary_html}</div><hr style='margin: 15px 0;'>"
            except Exception as e_reg:
                print(f"Warning: Regression failed for {ticker_stock}: {e_reg}")
                ax.text(0.5, 0.5, f'{ticker_stock}\nRegression error', ha='center', va='center', fontsize=10); ax.set_xticks([]); ax.set_yticks([]); continue

            ax.scatter(current_market_ex_ret_aligned, current_stock_ex_ret, color=scatter_color, label=f'ex{ticker_stock} (Actual)', s=35, alpha=0.5)
            ax.plot(current_market_ex_ret_aligned, model.fittedvalues, color=line_color, linewidth=1.5, label='Fitted values (SCL)')
            ax.set_xlabel(f'ex{market_ticker.replace("^","")} (Market Excess Return)', fontsize=10)
            if i % ncols == 0: ax.set_ylabel(f'Excess Return', fontsize=10) # Y-label for left-most plots
            ax.set_title(f'{ticker_stock}', fontsize=11)
            ax.tick_params(axis='both', which='major', labelsize=9)
            ax.legend(loc='lower right', fontsize=9, frameon=True, facecolor='white', framealpha=0.8, borderpad=0.5)
            # RULE 8.3: Use raw string for LaTeX
            eq_text = f'$\\alpha = {alpha_est*100:.2f}\\%$\n$\\beta = {beta_est:.2f}$\n$R^2 = {r_squared:.2f}$'
            ax.text(0.04, 0.96, eq_text, transform=ax.transAxes, fontsize=9, verticalalignment='top', bbox=dict(boxstyle='round,pad=0.4', fc='#F0E68C', alpha=0.85, edgecolor='#BDB76B'))
            
            ax.set_xlim(common_x_limits)
            # --- Fully Dynamic Y-axis limits ---
            all_y_values = np.concatenate([current_stock_ex_ret.values, model.fittedvalues])
            y_min_data = all_y_values.min()
            y_max_data = all_y_values.max()
            padding_y = (y_max_data - y_min_data) * 0.1 # 10% padding on range
            
            final_y_min = y_min_data - padding_y
            final_y_max = y_max_data + padding_y

            # Ensure 0 is visible if data range is near it or doesn't cross it significantly
            if final_y_min > -0.02 and final_y_max > 0 : final_y_min = min(final_y_min, -0.02)
            if final_y_max < 0.02 and final_y_min < 0 : final_y_max = max(final_y_max, 0.02)
            # Ensure a minimum span for very flat data
            if final_y_max - final_y_min < 0.1:
                 mid_point_y = (final_y_max + final_y_min) / 2
                 final_y_min = mid_point_y - 0.05
                 final_y_max = mid_point_y + 0.05
            ax.set_ylim(final_y_min, final_y_max)
        
        for j in range(num_stocks_to_plot, nrows * ncols): fig.delaxes(axes[j])
        
        suptitle_y_pos = 0.99 if nrows > 1 else 1.03 
        tight_layout_rect = [0, 0.02, 1, 0.95 if nrows > 1 else 0.90]

        fig.suptitle(f'Cost of Capital: Beta Estimation (Monthly Excess Returns, {start_date_dt.year}–{end_date_dt.year})', fontsize=16, y=suptitle_y_pos)
        plt.tight_layout(rect=tight_layout_rect)
        plt.show()

    with output_regression_stats_container:
        display(HTML(STATSMODELS_TABLE_CSS + regression_summaries_html_content))

# --- Link Button and Arrange UI ---
run_button.on_click(fetch_calculate_and_plot)
date_inputs_ui = widgets.HBox([start_date_widget, end_date_widget], layout=widgets.Layout(justify_content='center', margin='0 0 10px 0'))
ui_controls = widgets.VBox([date_inputs_ui, run_button], layout=widgets.Layout(align_items='center'))
full_ui = widgets.VBox([
    widgets.HTML("<h2 style='text-align:center;'>Cost of Capital: Beta Estimation Tool</h2>"),
    ui_controls,
    output_plots_container,
    widgets.HTML("<hr style='margin: 20px 0;'>"),
    output_regression_stats_container
])

# --- Initial Display ---
with warnings.catch_warnings():
    warnings.simplefilter("ignore", UserWarning) 
    display(full_ui)
    # fetch_calculate_and_plot() # Optionally run on load

---

### The Capital Asset Pricing Model

*   Historical Betas
    $$r_{i,t}^e = \alpha_i + \beta_i r_{M,t}^e + \epsilon_{i,t}$$
    $$\text{S.E.}(\hat{\beta}_i) = \frac{\sqrt{1-R^2}}{\sqrt{T}} \frac{\hat{\sigma}_i}{\hat{\sigma}_M}$$
*   The precision of an OLS beta estimate can be increased by
    *   Increasing the number of observations (T),
    *   By using portfolios instead of individual securities,
    *   By increasing the frequency of return observations

---



### The Capital Asset Pricing Model

*   Should idiosyncratic risk matter for pricing?

*   Would you ever buy a stock with negative expected returns?

*   How do the betas add up?

*   Can you arbitrage stocks that are on the SML?

*   Can you arbitrage stocks that are not on the SML?

---



### Question 1

The Capital Asset Pricing Model (CAPM) concludes that the expected return of a security is determined The Capital Asset Pricing Model (CAPM) concludes that the expected return of a security is determined by its systematic risk (beta), not its total risk (standard deviation). 

Yet, when constructing an optimal risky portfolio using the Markowitz model, the full covariance matrix (which includes total risk and correlations) is required. Explain how these two ideas are reconciled. 

Why does an individual security's firm-specific risk matter for portfolio construction but not for its expected return in a CAPM equilibrium?

---


### Question 1: Answer

*   The two ideas are reconciled through the concept of diversification. 
    - The Markowitz model is a portfolio construction tool that uses the full covariance matrix to build the most efficient portfolio by combining assets in a way that minimizes total risk for a given level of return. 
    - A key part of this process is minimizing the impact of firm-specific risks by combining assets whose unsystematic returns are not perfectly correlated.

*   The CAPM, however, is an equilibrium model that assumes all investors have already performed this optimization and hold the single, perfectly diversified market portfolio. In such a world, all firm-specific risk has been diversified away.

*   Firm-specific risk matters for portfolio construction because it is the "raw material" that diversification eliminates. 
    - The covariance between assets is critical for determining how effectively this risk can be reduced.

*   In the CAPM equilibrium, the marginal investor holds the market portfolio and is not exposed to any firm-specific risk. 
    - This diversifiable risk carries no market-wide exposure, it is not compensated with a higher expected return. 
    - Only systematic risk, which cannot be diversified away, warrants a risk premium.

---


### Question 2

A security analyst presents you with two stocks. 

- Stock A has a high R-squared of 0.80 in its single-index model regression, while Stock B has a low R-squared of 0.20. 

- The analyst claims that Stock A is a better investment because its returns are more predictable and closely track the market. 

- Critically evaluate this statement. 

  -   Does a high R-squared necessarily imply a superior investment or a higher expected return under the CAPM? 

  -   What does R-squared actually measure in this context?

---

### Question 2: Answer

* The analyst's statement is incorrect. 
    - A high R-squared does not necessarily imply a superior investment or a higher expected return.

* R-squared measures the proportion of a stock's total variance that is explained by the market's variance (i.e., its systematic risk).
    - Stock A (R² = 0.80) has 80% of its risk attributable to the market.
    
    - Stock B (R² = 0.20) has only 20% of its risk attributable to the market; the other 80% is firm-specific risk.

* According to the CAPM, expected return is a function of systematic risk (beta), not total risk or the proportion of risk that is systematic. 
    - Stock A could have a low beta and low expected return, while Stock B could have a high beta and high expected return. 
    - R-squared does not determine this.

* A high R-squared simply means the stock's returns are highly correlated with the market index, making it a good candidate for a "tracking" stock. 

* A low R-squared implies the stock's returns are driven more by firm-specific factors, offering greater potential for diversification benefits when added to a portfolio. Neither value, on its own, indicates whether a stock is a good or bad investment.

---

### Question 3

You are analyzing Stock X using the single-index model. You have the following information from a regression of Stock X's excess returns on the market's excess returns:

*   Beta ($\beta_X$) = 1.4
*   Alpha ($\alpha_X$) = 2.0%
*   R-squared ($R^2$) = 0.49
*   The standard deviation of the market index's excess returns is $\sigma_M = 20\%$.

    a. What is the standard deviation of Stock X's excess returns?
    
    b. What is the covariance between Stock X and the market index?
    
    c. What is the correlation coefficient between Stock X and the market index?

---

### Question 3: Answer

The R-squared is the ratio of systematic variance to total variance:
$$
R^2 = \frac{\beta_X^2 \sigma_M^2}{\sigma_X^2} = 0.49
$$
Rearrange this to solve for the total variance of Stock X, $\sigma_X^2$:
$$
\sigma_X^2 = \frac{\beta_X^2 \sigma_M^2}{R^2} = \frac{(1.4)^2 \times (0.20)^2}{0.49} = \frac{1.96 \times 0.04}{0.49} = 0.16
$$

$$
\sigma_X = \sqrt{0.16} = 0.40 = 40.00\%
$$

$$
\text{Cov}(R_X, R_M) = \beta_X \sigma_M^2 \quad \Rightarrow \quad \text{Cov}(R_X, R_M) = 1.4 \times (0.20)^2 = 1.4 \times 0.04 = 0.056$$

$$\rho_{X,M} = \frac{\text{Cov}(R_X, R_M)}{\sigma_X \sigma_M} \quad \Rightarrow \quad \rho_{X,M} = \frac{0.056}{0.40 \times 0.20} = \frac{0.056}{0.08} = 0.70$$

Alternatively, the square root of R-squared is the correlation coefficient: $\sqrt{0.49} = 0.70$.

---

In [8]:
import numpy as np

# Given data
beta_X = 1.4
alpha_X = 0.02  # Not used, but included for completeness
R_squared = 0.49
sigma_M = 0.20

# Part a: Standard deviation of Stock X's excess returns
systematic_variance = beta_X**2 * sigma_M**2
sigma_X_squared = systematic_variance / R_squared
sigma_X = np.sqrt(sigma_X_squared)
print(f"Part a: Standard Deviation of Stock X (sigma_X): {sigma_X * 100:.2f}%")

# Part b: Covariance between Stock X and market
cov_XM = beta_X * sigma_M**2
print(f"Part b: Covariance (Cov(R_X, R_M)): {cov_XM:.3f}")

# Part c: Correlation coefficient
rho_XM = cov_XM / (sigma_X * sigma_M)
rho_XM_alt = np.sqrt(R_squared)
print(f"Part c: Correlation Coefficient (rho_X,M): {rho_XM:.2f}")
print(f"Part c (Alternative): sqrt(R-squared): {rho_XM_alt:.2f}")

Part a: Standard Deviation of Stock X (sigma_X): 40.00%
Part b: Covariance (Cov(R_X, R_M)): 0.056
Part c: Correlation Coefficient (rho_X,M): 0.70
Part c (Alternative): sqrt(R-squared): 0.70


---

### Question 4

An analyst provides you with the following single-index model regression output for two stocks, A and B, against the S&P 500 (M):

| Stock | Alpha ($\alpha$) | Beta ($\beta$) | Residual Std. Dev. ($\sigma_e$) |
| :---- | :--------------- | :------------- | :------------------------------ |
| A     | 1.5%             | 1.2            | 8%                              |
| B     | -0.5%            | 0.8            | 6%                              |

The expected excess return of the S&P 500 is 8%, and its standard deviation is 20%.

- What are the expected excess returns for Stock A and Stock B?

- What is the covariance between the returns of Stock A and Stock B?

- What would be the alpha, beta, and residual standard deviation of an equally weighted portfolio of Stock A and Stock B?

---

### Question 4: Answer

The expected excess return for a stock under the single-index model is given by $E(R_i) = \alpha_i + \beta_i E(R_M)$.
$$
E(R_A) = 1.5\% + 1.2 \times 8\% = 11.10\% \quad \text{and} \quad E(R_B) = -0.5\% + 0.8 \times 8\% = 5.90\%$$

Under the single-index model, the covariance between two stocks is the product of their betas and the market variance.
$$
\text{Cov}(R_A, R_B) = \beta_A \beta_B \sigma_M^2 = 1.2 \times 0.8 \times (0.20)^2 = 0.0384
$$

$$
\alpha_P = w_A\alpha_A + w_B\alpha_B = (0.5 \times 1.5\%) + (0.5 \times -0.5\%) = 0.50\%
$$
$$
\beta_P = w_A\beta_A + w_B\beta_B = (0.5 \times 1.2) + (0.5 \times 0.8) = 1.00
$$

The variance of the portfolio's residual is the weighted average of the individual residual variances (since the residuals are uncorrelated by assumption).

$$
\sigma^2(e_P) = w_A^2\sigma^2(e_A) + w_B^2\sigma^2(e_B) = (0.5)^2(0.08)^2 + (0.5)^2(0.06)^2 = 0.0025
$$

The residual standard deviation of the portfolio is the square root of this variance.
$$
\sigma(e_P) = \sqrt{0.0025} = 5.00\%
$$

---

In [9]:
import numpy as np

# Given data
alpha_A = 0.015
beta_A = 1.2
sigma_e_A = 0.08
alpha_B = -0.005
beta_B = 0.8
sigma_e_B = 0.06
E_rm_excess = 0.08
sigma_M = 0.20
w_A = 0.5
w_B = 0.5

# Part a: Expected excess returns
E_rA_excess = alpha_A + beta_A * E_rm_excess
E_rB_excess = alpha_B + beta_B * E_rm_excess
print(f"Part a: E(R_A excess): {E_rA_excess * 100:.2f}%")
print(f"Part a: E(R_B excess): {E_rB_excess * 100:.2f}%")

# Part b: Covariance between A and B
cov_AB = beta_A * beta_B * sigma_M**2
print(f"Part b: Cov(R_A, R_B): {cov_AB:.4f}")

# Part c: Portfolio alpha, beta, residual std dev
alpha_P = w_A * alpha_A + w_B * alpha_B
beta_P = w_A * beta_A + w_B * beta_B
var_e_P = (w_A**2 * sigma_e_A**2) + (w_B**2 * sigma_e_B**2)
sigma_e_P = np.sqrt(var_e_P)
print(f"Part c: Alpha_P: {alpha_P * 100:.2f}%")
print(f"Part c: Beta_P: {beta_P:.2f}")
print(f"Part c: Sigma_e_P: {sigma_e_P * 100:.2f}%")

Part a: E(R_A excess): 11.10%
Part a: E(R_B excess): 5.90%
Part b: Cov(R_A, R_B): 0.0384
Part c: Alpha_P: 0.50%
Part c: Beta_P: 1.00
Part c: Sigma_e_P: 5.00%


### Question 5

You are a portfolio manager evaluating a stock, "Kappa Corp," for potential mispricing using the CAPM. You have the following data and forecasts:

*   Kappa Corp's estimated beta: 1.5
*   Kappa Corp's residual standard deviation, $\sigma(e)$: 15%
*   Your forecast for Kappa Corp's return: 16%
*   Your forecast for the market return: 10%
*   The market's standard deviation, $\sigma_M$: 20%
*   The current risk-free rate: 2%

a) According to the CAPM, what is the required rate of return for Kappa Corp?

b) What is the alpha of Kappa Corp based on your forecasts?

c) You decide to form an optimal active portfolio by combining Kappa Corp with the passive market index. Calculate the optimal weight to assign to Kappa Corp ($w_A^*$) in this active portfolio.

d) What is the beta of this new optimal active portfolio?

e) An analyst claims that because the new active portfolio's beta is higher than the market's beta of 1.0, its Sharpe ratio must be lower than the market's. Is the analyst correct? Calculate the Sharpe Ratio of your new active portfolio to verify your answer.

---

### Question 5: Answer

$$
E(r_{\text{Kappa}}) = r_f + \beta_{\text{Kappa}}[E(r_M) - r_f] \quad \Rightarrow \quad E(r_{\text{Kappa}}) = 2\% + 1.5 \times (10\% - 2\%) = 2\% + 1.5 \times 8\% = 14.00\%
$$

Alpha is the difference between your forecast return and the required return predicted by the CAPM.
$$
\alpha_{\text{Kappa}} = \text{Forecast } E(r) - \text{CAPM } E(r) \quad \Rightarrow \quad \alpha_{\text{Kappa}} = 16\% - 14\% = 2.00\%$$

The optimal weights for the active portfolio (A) and the passive market index (M) are found by first calculating the ratio of their "alpha-to-residual-variance" and "market-premium-to-variance":

$$ \frac{w_A}{w_M} = \frac{\alpha_A / \sigma^2(e_A)}{E(R_M) / \sigma_M^2} = \frac{0.02 / (0.15)^2}{0.08 / (0.20)^2} = \frac{0.8889}{2.0} = 0.4444$$

$$ \text{Since} \quad w_A + w_M = 1 \quad \Rightarrow \quad w_A = \frac{0.4444}{1 + 0.4444} = 0.3077 = 30.77\% $$

$$
w_M = 1 - w_A = 69.23\%
$$
The optimal risky portfolio consists of a 30.77% allocation to a beta-neutral active portfolio (built by taking a long position in Kappa Corp and a short position in the market index) and a 69.23% allocation to the passive market index.

d)
The active portfolio is constructed to be beta-neutral (beta of 0). The beta of the final combined risky portfolio is the weighted average of its components:
$$
\beta_P = (w_A \times \beta_{\text{active}}) + (w_M \times \beta_M) = (0.3077 \times 0) + (0.6923 \times 1.0) = 0.6923
$$
e)
The analyst claims that a portfolio with beta different from 1.0 must have a lower Sharpe ratio than the market. The analyst is incorrect.

The squared Sharpe Ratio of the new active portfolio is the sum of the market's squared Sharpe Ratio and the squared Information Ratio of the active asset.
$$
S_P^2 = S_M^2 + \left(\frac{\alpha_A}{\sigma(e_A)}\right)^2
$$
$$
S_P^2 = (0.40)^2 + \left(\frac{0.02}{0.15}\right)^2 = 0.16 + 0.0178 = 0.1778
$$
$$
S_P = \sqrt{0.1778} = 0.4216
$$
Since $S_P (0.4216) > S_M (0.40)$, the active portfolio is superior. The improvement comes from the alpha generated by the market-neutral active position.

---

In [10]:
import numpy as np

# Given data
beta = 1.5
forecast_r = 0.16
E_rm = 0.10
rf = 0.02
alpha = forecast_r - (rf + beta * (E_rm - rf))  # For verification
sigma_e = 0.15
sigma_M = 0.20
E_rm_excess = E_rm - rf  # 0.08

# Part a: Required return
required_r = rf + beta * (E_rm - rf)
print(f"Part a: Required Return: {required_r * 100:.2f}%")

# Part b: Alpha
alpha_calc = forecast_r - required_r
print(f"Part b: Alpha: {alpha_calc * 100:.2f}%")

# Part c: Optimal w_A*
alpha_over_var_e = alpha_calc / sigma_e**2
market_ratio = E_rm_excess / sigma_M**2
ratio = alpha_over_var_e / market_ratio
w_A = ratio / (1 + ratio)
w_M = 1 - w_A
print(f"Part c: Optimal w_A*: {w_A * 100:.2f}%")

# Part d: Portfolio Beta
beta_active = 0
beta_P = w_A * beta_active + w_M * 1
print(f"Part d: Beta_P: {beta_P:.4f}")

# Part e: Sharpe Ratio
S_m = E_rm_excess / sigma_M
info_ratio_sq = (alpha_calc / sigma_e)**2
S_p_sq = S_m**2 + info_ratio_sq
S_p = np.sqrt(S_p_sq)
print(f"Part e: S_m: {S_m:.2f}")
print(f"Part e: S_p: {S_p:.4f}")

Part a: Required Return: 14.00%
Part b: Alpha: 2.00%
Part c: Optimal w_A*: 30.77%
Part d: Beta_P: 0.6923
Part e: S_m: 0.40
Part e: S_p: 0.4216


---

### What is next?

*   Multifactor Models
*   Arbitrage Pricing Theory
*   Reading(s): BKM Ch. 10
*   Suggested Problems
    *   Ch.8: 6, 9-12
    *   Ch.9: 2, 4, 8, 17-19, 21.
    *   Ch.9-CFA Problems: 1-2, 8-9,12.

---
